In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time
from time import sleep

In [2]:
chromedriver_path = r"C:/Users/Aymeric Leboucher/OneDrive - De Vinci/ESILV/A5/Webscrapping/TP2/chromedriver.exe"

In [3]:
def get_page_source(link):
    driver = webdriver.Chrome(chromedriver_path)
    driver.get(link)
    src=driver.page_source
    return src

In [4]:
def page_parser(page):
    parser = BeautifulSoup(page, "lxml")
    table = parser.find("div", attrs={"class":"chart-results-list // lrv-u-padding-t-150 lrv-u-padding-t-050@mobile-max"})
    return table

In [5]:
to_remove_from_song_titles = [ 'Imprint/Promotion Label:',
 'Songwriter(s):',
 'Producer(s):']

def extract_song_titles(table):
    song_titles = table.find_all("h3")
    ori_songlist = [h.text.strip() for h in song_titles[2:]]
    songlist = [word for word in ori_songlist if word not in to_remove_from_song_titles]
    return songlist

In [6]:
def extract_artists(table):
    to_remove_from_artists = ["",'Share Chart on Twitter','Share Chart on Embed', 'This Week',
                            'Award',
                            'Last Week',
                            'Peak Pos.',
                            'Wks on Chart',
                            'Twitter',
                            'Share Chart on Copy Link',
                            'Copy Link',
                            'Share Chart on Facebook',
                            'Facebook',
                            'Embed',
                            'RIAA Certification:',
                            'Diamond',
                            "Platinum",
                            'Platinum x2',
                            'Platinum x3',
                            'Platinum x4',
                            'Platinum x5',
                            'RE-\nENTRY',
                            'Gold',
                            'NEW',
                            "-"]

    artists = table.find_all("span")
    artists = [h.text.strip() for h in artists]
    artists = [artists[i] for i in range(len(artists)) if artists[i] not in to_remove_from_artists if not artists[i].isdigit()]
    return artists 

In [7]:
def convert_to_top_100(songlist, artists):
    top_100 = pd.concat([pd.Series(songlist), pd.Series(artists)], axis=1)
    top_100.rename(columns = {0:"Song_Title", 1:"Artist"}, inplace=True)
    return top_100

In [8]:
def get_top_100(year, month, day):
    month = str(month)
    if len(month)==1:
        month = "0" + month
    day = str(day)
    if len(day)==1:
        day = "0" + day
    print(year, "/", month, "/", day)
    link = f"https://www.billboard.com/charts/hot-100/{year}-{month}-{day}/"
    src = get_page_source(link)
    table = page_parser(src)
    songlist = extract_song_titles(table)
    artists = extract_artists(table)
    top_100 = convert_to_top_100(songlist, artists)
    return top_100

In [9]:
get_top_100(1989, 2, 10)

1989 / 02 / 10


,Song_Title,Artist
0,Straight Up,Paula Abdul
1,When I'm With You,Sheriff
2,Wild Thing,Tone-Loc
3,Born To Be My Baby,Bon Jovi
4,When The Children Cry,White Lion
...,...,...
95,Imagine,Tracie Spencer
96,Welcome To The Jungle,Guns N' Roses
97,Left To My Own Devices,Pet Shop Boys
98,Finish What Ya Started,Van Halen
